In [ ]:
#%pip install pytesseract

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Imports

In [3]:
import pytesseract

###  Optical Character Recognition (OCR): 
To extract text from scanned contracts using Tesseract